In [9]:
from pathlib import Path
import sys
import pandas as pd
from IPython.display import display, HTML
from IPython.display import display, HTML
import re
from datetime import datetime, timezone

# Set project root to one level above current notebook directory
project_root = Path().resolve().parent
sys.path.append(str(project_root))

from utils.db_connection import get_db_connection

```json
{
  "posts": {
    "post_id": "VARCHAR",
    "subreddit_id": "VARCHAR",
    "title": "TEXT",
    "selftext": "TEXT",
    "created_utc": "INTEGER",
    "score": "INTEGER",
    "num_comments": "INTEGER",
    "permalink": "TEXT"
  }
}
```

In [7]:
db = get_db_connection()
df = pd.read_sql_query(
    """
    SELECT p.*, s.name AS subreddit_name
    FROM posts p
    LEFT JOIN subreddits s ON p.subreddit_id = s.subreddit_id
    """, db)
db.close()

# Convert creation timestamp to datetime
df['created_at'] = pd.to_datetime(df['created_utc'], unit='s')

# Compute summary metrics
total_posts = len(df)
unique_subs = df['subreddit_name'].nunique()
min_date = df['created_at'].min().strftime('%Y-%m-%d')
max_date = df['created_at'].max().strftime('%Y-%m-%d')

# Determine top 10 subreddits by post count
top_subs = df['subreddit_name'].value_counts().head(10)

# Build markdown summary
md = (
    "**Snapshot Data Notice**  \n"
    "Data shown is a snapshot as of the last database update, not real-time.\n\n"
    f"**Total posts**: {total_posts:,}  \n"
    f"**Unique subreddits**: {unique_subs:,}  \n"
    f"**Time range**: {min_date} to {max_date}\n\n"
    "**Top 10 Subreddits by Post Count**  \n"
)
for name, count in top_subs.items():
    md += f"- {name}: {count:,}\n"

# Display summary
display(Markdown(md))


**Snapshot Data Notice**  
Data shown is a snapshot as of the last database update, not real-time.

**Total posts**: 18,907  
**Unique subreddits**: 51  
**Time range**: 2014-12-28 to 2025-07-16

**Top 10 Subreddits by Post Count**  
- WGU: 4,561
- wguaccounting: 1,674
- WGUCyberSecurity: 1,309
- WGU_MBA: 1,276
- wgueducation: 1,163
- WGU_CompSci: 1,160
- WGUTeachersCollege: 1,160
- wgu_devs: 1,137
- WGU_MSDA: 1,108
- WGUIT: 1,105


## What are people talking about today?

```json
{
  "posts": {
    "post_id": "VARCHAR",
    "subreddit_id": "VARCHAR",
    "title": "TEXT",
    "selftext": "TEXT",
    "created_utc": "INTEGER",
    "score": "INTEGER",
    "num_comments": "INTEGER",
    "permalink": "TEXT"
  }
}
```

In [36]:
# Filter posts from the last 24 hours
latest_timestamp = df['created_at'].max()
df = df[df['created_at'] >= latest_timestamp - pd.Timedelta(hours=24)].copy()
print(f"✅ {len(df)} posts found")
display(df[['post_id', 'title', 'selftext', 'subreddit_name','created_utc']].head(5).reset_index(drop=True))

✅ 63 posts found


,post_id,title,selftext,subreddit_name,created_utc
0,1lzfct7,MSN Application,I am in the process of filling out the applica...,WGU,1752475215
1,1lzf7hc,It is done.,Honestly one of the hardest things I've ever d...,WGU,1752474652
2,1lzafb3,Completed my first class!!!!,"Y’all my start date was July 1st, and I just c...",WGU,1752458805
3,1lzab83,Accounting at WGU,Anyone got a degree from WGU in accounting and...,WGU,1752458464
4,1lza8f0,WGU Commencement,So I am finishing my degree this month and wan...,WGU,1752458245


In [10]:
def time_ago_from_utc(utc_ts):
    now = datetime.now(timezone.utc)
    created_time = datetime.fromtimestamp(utc_ts, tz=timezone.utc)
    delta = now - created_time

    seconds = int(delta.total_seconds())
    if seconds < 60:
        return f"{seconds} seconds ago"
    elif seconds < 3600:
        return f"{seconds // 60} minutes ago"
    elif seconds < 86400:
        return f"{seconds // 3600} hours ago"
    else:
        return f"{seconds // 86400} days ago"

df['time_ago'] = df['created_utc'].apply(time_ago_from_utc)
display(df[['post_id', 'title', 'selftext', 'subreddit_name','time_ago']].head(5).reset_index(drop=True))

,post_id,title,selftext,subreddit_name,time_ago
0,1k6jeqd,Examity,I’m curious as to how examity works. I read on...,WGU,83 days ago
1,1k6j88n,Any Canadians here pursuing software developme...,I’m considering getting a software development...,WGU,83 days ago
2,1k6iufu,ANYONE IN D277,I’m half way through Front End Web Development...,WGU,83 days ago
3,1k6hw8z,DING!,Finally!! \n\n \nIt's been a rough 2 years fo...,WGU,83 days ago
4,1k6gjrk,Anyone ever have a capstone returned for revis...,,WGU,83 days ago


In [16]:
# Posts by subreddit
print("Posts by subreddit:")
for name, count in df['subreddit_name'].value_counts().items():
    print(f"{name}: {count}")



Posts by subreddit:
WGU: 34
wguaccounting: 7
wgueducation: 5
WGU_MBA: 4
WGUTeachersCollege: 3
WGU_Business: 2
WGU_CompSci: 2
wgu_devs: 2
WGUIT: 1
WGU_CSA: 1
WGU_Military: 1
wgu_employees: 1


In [24]:
# %%
# Add category to DataFrame
wgu_df['category_num'] = wgu_df['post_id'].map(post_category_map)
wgu_df['category'] = wgu_df['category_num'].map(categories)

# Display posts by category
for cat_num, cat_name in categories.items():
    cat_posts = wgu_df[wgu_df['category_num'] == cat_num]
    if not cat_posts.empty:
        print(f"\n{cat_name} posts:")
        display(
            cat_posts[['post_id', 'text_length', 'preview']].reset_index(drop=True).style.set_properties(**{
                'white-space': 'nowrap',
                'text-overflow': 'ellipsis',
                'overflow': 'hidden',
            })
        )


Course Help & Tips posts:


,post_id,text_length,preview
0,1lz3gq2,859,"Finishing individual courses after the degree plan “deadline” (but still within the term)? Hi all, I’m working on an ..."
1,1lz59ae,802,"C190 issues with academic plan to retake OA So I barely failed my first OA. Thought I'd just retake it but no, appare..."
2,1lz5698,541,"D077 Concepts in Marketing, Sales and Customer Contact tips Hi everyone. I just recently started this class and I’m ..."
3,1lz22mz,514,Fundamentals of Spreadsheets D388 | Tips | Passed in 2 days **Difficulty: 3/10** **Tips:** \-You only need the vi...
4,1lyvqxh,482,D028 task not showing completed Hi! Has anyone had this issue with other classes? I finished an assignment that was o...
5,1lz8cyy,386,"Passed ITIL Honestly was worried for no reason. I studied probably 30 minutes the day of, nothing prior. Should have ..."
6,1lz5o1s,295,"D196 and D076 For anyone who hasn’t taken D196 and D076 , my mentor just informed me that those 2 classes will be upd..."
7,1lz14z6,177,What study materials did you guys use for C955 I saw something about cohorts and Khan Academy. Any specific videos th...
8,1lyxphy,104,Are there any cohort videos or study guides for D076? There's nothing under course tips or course search...
9,1lz7nws,97,Healthcare Ecosystems Who here has taken Healthcare Ecosystems D391? What did you think about it?...



Degree Planning & Program Selection posts:


,post_id,text_length,preview
0,1lz0fej,1152,Military officer route with WGU degree? Going for an accounting degree at WGU and was curious how the military and it...
1,1lz7p1x,860,"Sophia learning then wgu. Any advice? Hi everybody, I (28f) am looking into school options. I've been told about wgu ..."
2,1lyxm5s,606,WGU BS Cybersecurity or Business Management?? I am currently enrolled in WGU for Cybersecurity but I am unsure if I s...
3,1lyrmu1,380,"Masters degree? Hello, I currently will have a BA degree in criminal justice. I am interested in getting into tech bu..."
4,1lz61qe,358,I have a question about degrees I’ve been juggling going to do cybersecurity (I have three years of experience in IT ...
5,1lyzarj,308,"Precalculus or Quantitative Lit? Those that have taken WGUs Quantitative Literacy class, was it easier? I’m strugglin..."
6,1lz5vj0,240,Exams? I know WGU has proctored exams but does every degree require exams? I’m looking to get my bachelors in busines...
7,1lzab83,118,Accounting at WGU Anyone got a degree from WGU in accounting and landed a job? Just curious what their experience was...
8,1lyqmcr,117,Online degree Will anyone be able to tell such as another university or employer that I have done an “online” degree?...
9,1lz3lx4,93,What is the likelihood that a 18 year old straight out of high school gets accepted into WGU?...



Milestones & Motivation posts:


,post_id,text_length,preview
0,1lzf7hc,891,It is done. Honestly one of the hardest things I've ever done for myself. Did terrible in highschool (was diagnosed a...
1,1lz7jcg,451,"Almost there! I have two classes left. A total of four tasks, I’ve already submitted two of them. I told my five year..."
2,1lyym2h,359,Struggling with motivation I really killed it with my first term and knocked out a lot of classes but right now I’ve ...
3,1lzafb3,121,"Completed my first class!!!! Y’all my start date was July 1st, and I just completed my 1st class!!! 🥳 I’m so so excit..."
4,1lz8psk,39,A wins a win🙏 C949 passed first attempt...
5,1lz61qj,26,I finally got my confetti!...



Teaching & Education Programs posts:


,post_id,text_length,preview
0,1lyuzmn,975,"For Master of Education, Ed Tech and Instructional Design, are there any ""Stand Alone"" courses recommended to do whil..."
1,1lyylv7,493,"WGU D660- GoReact Hi all, I’m slightly confused on how to do my GoReact for Instructional Technology and Online Pedag..."



Student Life & Logistics posts:


,post_id,text_length,preview
0,1lz2b0n,430,"I feel bad for skipping days due to being sick Went on vacation, came back, bam I have covid lol. I feel so under the..."
1,1lynr48,336,Withdrawn So I’ve been academically withdrawn unfortunately. Life’s had plenty of courses with me so I didn’t get to ...
2,1lyxgpp,284,Grad hat The button on the grad cap is definitely off center 😂 but it was $5 from Amazon. And it will only be worn fo...
3,1lza8f0,272,"WGU Commencement So I am finishing my degree this month and want to attend an in-person commencement, but all of the ..."
4,1lzfct7,180,MSN Application I am in the process of filling out the application for MSN and it won’t let me pass the employment pa...
5,1lyzi01,100,"Dr.Ph, what positions do you hold and how much is your salary? Trying to see if it’s worth pursuing...."


In [22]:
# %%
# Define categories as a dictionary
categories = {
    1: "Course Help & Tips",
    2: "Degree Planning & Program Selection",
    3: "Milestones & Motivation",
    4: "Teaching & Education Programs",
    5: "Student Life & Logistics"
}


post_category_map = {
    "1lz0fej": 2,  # Military officer route with WGU degree?
    "1lyuzmn": 4,  # Master of Education, Ed Tech
    "1lzf7hc": 3,  # It is done. Personal story of achievement
    "1lz7p1x": 2,  # Sophia learning then WGU - planning path
    "1lz3gq2": 1,  # Finishing courses past deadline - logistics/course
    "1lz59ae": 1,  # C190 issues with retake
    "1lyxm5s": 2,  # Choosing between Cybersecurity or Business Management
    "1lz5698": 1,  # D077 class tips
    "1lz22mz": 1,  # Spreadsheet course tips
    "1lyylv7": 4,  # GoReact confusion - Teaching tools
    "1lyvqxh": 1,  # D028 task not completing
    "1lz7jcg": 3,  # Almost finished - motivational
    "1lz2b0n": 5,  # Feeling bad about skipping days - student life
    "1lz8cyy": 1,  # Passed ITIL - exam tips
    "1lyrmu1": 2,  # Master's degree planning
    "1lyym2h": 3,  # Struggling with motivation
    "1lz61qe": 2,  # Degree decision: Cybersecurity
    "1lynr48": 5,  # Academic withdrawal - personal logistics
    "1lyzarj": 2,  # Precalc vs Quant Lit - course selection
    "1lz5o1s": 1,  # D196/D076 update info
    "1lyxgpp": 5,  # Graduation cap - student life
    "1lza8f0": 5,  # WGU Commencement
    "1lz5vj0": 2,  # Exams per degree - planning question
    "1lzfct7": 5,  # MSN application issue
    "1lz14z6": 1,  # Study materials for C955
    "1lzafb3": 3,  # Completed first class - motivational
    "1lzab83": 2,  # Accounting degree outcome
    "1lyqmcr": 2,  # Online degree reputation
    "1lyxphy": 1,  # Study guides for D076
    "1lyzi01": 5,  # Dr.Ph career/salary question - logistics
    "1lz7nws": 1,  # Healthcare Ecosystems course feedback
    "1lz3lx4": 2,  # High school acceptance into WGU
    "1lz8psk": 3,  # Passed C949
    "1lz61qj": 3   # Got confetti - celebration
}
# %%
# Count categorized posts
num_categorized = len(post_category_map)
print(f"✅ {num_categorized} posts categorized")

✅ 34 posts categorized


In [28]:
# %%
def sentiment_style(val):
    if val >= 0.5:
        return 'color: green'
    elif val <= -0.5:
        return 'color: red'
    else:
        return 'color: gray'

def sentiment_emoji(val):
    if val >= 0.5:
        return '🟢'
    elif val <= -0.5:
        return '🔴'
    else:
        return '⚪️'

# Add emoji column
wgu_df['VADER_emoji'] = wgu_df['VADER'].apply(sentiment_emoji)

# Display with styling
display(
    wgu_df[['post_id', 'text_length', 'preview', 'VADER', 'VADER_emoji']]
    .sort_values(by='VADER', ascending=False)
    .reset_index(drop=True)
    .style
    .applymap(sentiment_style, subset=['VADER'])
    .set_properties(subset=['preview'], **{
        'white-space': 'nowrap',
        'text-overflow': 'ellipsis',
        'overflow': 'hidden',
    })
)

/var/folders/4k/nn6_64p916g2s6wytgx_n4rr0000gn/T/ipykernel_90972/967059948.py:27: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(sentiment_style, subset=['VADER'])


,post_id,text_length,preview,VADER,VADER_emoji
0,1lz8psk,39,A wins a win🙏 C949 passed first attempt...,0.817600,🟢
1,1lzafb3,121,"Completed my first class!!!! Y’all my start date was July 1st, and I just completed my 1st class!!! 🥳 I’m so so excit...",0.581500,🟢
2,1lyzarj,308,"Precalculus or Quantitative Lit? Those that have taken WGUs Quantitative Literacy class, was it easier? I’m strugglin...",0.487100,⚪️
3,1lz3lx4,93,What is the likelihood that a 18 year old straight out of high school gets accepted into WGU?...,0.458800,⚪️
4,1lyrmu1,380,"Masters degree? Hello, I currently will have a BA degree in criminal justice. I am interested in getting into tech bu...",0.401900,⚪️
5,1lz0fej,1152,Military officer route with WGU degree? Going for an accounting degree at WGU and was curious how the military and it...,0.318200,⚪️
6,1lzab83,118,Accounting at WGU Anyone got a degree from WGU in accounting and landed a job? Just curious what their experience was...,0.318200,⚪️
7,1lyxgpp,284,Grad hat The button on the grad cap is definitely off center 😂 but it was $5 from Amazon. And it will only be worn fo...,0.261700,⚪️
8,1lyzi01,100,"Dr.Ph, what positions do you hold and how much is your salary? Trying to see if it’s worth pursuing....",0.226300,⚪️
9,1lza8f0,272,"WGU Commencement So I am finishing my degree this month and want to attend an in-person commencement, but all of the ...",0.038700,⚪️


In [ ]:
<.3 + '?'  = help seeking

In [ ]:
# %%
# Reverse lookup: category number to post IDs
from collections import defaultdict

category_posts = defaultdict(list)
for post_id, cat_num in post_category_map.items():
    if cat_num:
        category_posts[categories[cat_num]].append(post_id)

# Print each category and its posts
for category_name, post_ids in category_posts.items():
    print(f"\n{category_name} posts:")
    for pid in post_ids:
        row = wgu_df[wgu_df['post_id'] == pid].iloc[0]
        text = row['text'].replace('\n', ' ').strip()
        preview = text[:117] + '...' if len(text) > 120 else text
        print(f"{pid} | {row['text_length']} chars | {preview}")

In [4]:
# %%
from IPython.display import display, Markdown
import pandas as pd

# Helper to truncate long text
def truncate(text, max_len):
    if not isinstance(text, str):
        return ''
    return text if len(text) <= max_len else text[:max_len].rstrip() + "..."

# Top Posts by Score
display(Markdown("## Top Posts by Score"))

top_alltime = df.nlargest(10, 'score')
table = pd.DataFrame({
    'Title': top_alltime['title'].apply(lambda x: truncate(x, 100)),
    'Self-text': top_alltime['selftext'].apply(lambda x: truncate(x, 200)),
    'Score': top_alltime['score'],
})

styled = (
    table.style
         .hide(axis='index')
         .set_properties(subset=['Title','Self-text'], **{'text-align':'left','white-space':'normal'})
         .format({'Score':'{:,}'})
)

display(styled)

## Top Posts by Score

Title,Self-text,Score
I have no one to tell this to...,"I did it, guys!! 🎓 I officially finished my Bachelor’s degree in just 2 months and 13 days! I transferred 63 credits from community college and had only 57 credits left to complete. Every one of us...","1,605"
I did it,Started in February and finished mid May. I never thought I could do it. I transferred a bunch of GE from Sophia. I struggled with ethics of technology and got lucky that they changed it to a PA. The...,"1,454"
I don’t have anyone to share with but I finally did it!!,"Just before I turn 30! If I can do it, so can you! 🥳","1,066"
How I Got a Bachelors Degree for $793 in 8 Months with a Newborn and Full-Time Job,"TLDR: All core classes through Sophia, 2 months for $200. Rest of degree at WGU, 6 months for $593 (thanks to Pell Grant). Did this while working full-time and co-parenting full-time (while my partner...",912
Graduated in March and just accepted an offer!!!!,So before WGU I went to community college and dropped out twice. Once I found out I was going to be a father (I was 22) I immediately went back to community college and graduated while working full ti...,789
Owl Done!,Just got my diploma in the mail today! Took about 3 weeks give or take. This degree has already helped me with landing a job and I hope it can do the same for you guys too. I’ll actually be doing one...,745
JUST RECEIVED A JOB OFFER!,"HEY EVERYONE! Just got an offer for a $60,000/year position and I’m beyond excited! The job search journey since March after the Dallas commencement has been intense. I sent out hundreds of applicatio...",733
Graduated in a Year 💪🏼 Ask Anything,"Did this while full time job as a chef, no kids, have a wifey. I transferred only one class, ITF+ to qualify because my GPA wasn’t good enough due to not caring in school. Career changer from a chef...",686
"After going back to school 4 separate times, I finally have my Bachelor's degree.","Hello all, just wanted to share my journey in case anyone else can relate. I went to a 4-year university straight out of high school despite not knowing what I wanted to do with my life at all. I only...",652
BS in 3.5 months!!!,I started January 1st and I am so happy to have just passed my final class! Finishing in one term was my goal from the start but many people told me it was unrealistic. I work 50 hours a week and made...,640


In [5]:
# %%
from IPython.display import display, Markdown
import pandas as pd

# Top Posts by Number of Comments
display(Markdown("## Top Posts by Number of Comments"))

top_comments = df.nlargest(10, 'num_comments')
table_comments = pd.DataFrame({
    'Title':      top_comments['title'].apply(lambda x: truncate(x, 100)),
    'Self-text':  top_comments['selftext'].apply(lambda x: truncate(x, 200)),
    'Comments':   top_comments['num_comments'],
})

styled_comments = (
    table_comments.style
                  .hide(axis='index')
                  .set_properties(subset=['Title','Self-text'], **{'text-align':'left','white-space':'normal'})
                  .format({'Comments': '{:,.0f}'})
)

display(styled_comments)

## Top Posts by Number of Comments

Title,Self-text,Comments
How are you guys paying for WGU?,"I know WGU is already very affordable compared to other universities but still, money is very tight. Are there any tips or methods you guys have to bring down the cost of tuition before I apply for en...",359
I have no one to tell this to...,"I did it, guys!! 🎓 I officially finished my Bachelor’s degree in just 2 months and 13 days! I transferred 63 credits from community college and had only 57 credits left to complete. Every one of us...",319
BSIT WGU STUDENTS!!!,"WGU BSIT Students!!! Hey all, so I am looking for currently enrolled students in the BSIT degree program at WGU. I made a discord designed for students in this program. It will be a server designed t...",301
Graduated in a Year 💪🏼 Ask Anything,"Did this while full time job as a chef, no kids, have a wifey. I transferred only one class, ITF+ to qualify because my GPA wasn’t good enough due to not caring in school. Career changer from a chef...",240
I wanna quit so fckin bad,I'm so close yet so far to being done with my bachelors in IT management. 32 credits left but i barely passed Project management after a month of studying and have these massive pain in the ass class...,188
(UPDATED) 11/2023 Software Security and Testing D385,"**Disclaimer: I have never posted before, ever in my life,** this is my first ever reddit post, but I felt that people need to hear the truth about this class!! ​ I passed my OA and felt over...",174
Is 52 Too Old? Should I Bother?,I know technically it’s not but realistically? I’m currently a Sr Application Support Analyst for my local government’s IT Dept. I’m in charge of about 10 servers and their applications despite nothin...,170
Am I the only one?!?!,"I am so burnt out, I just made an appointment with a career counselor, my mentor, and my instructor. I get that some (ALOT) of students fly through this stuff (literal hours vs me weeks or months) and...",166
WTF is this?!,Context: didn’t realize the “logmein/applet” software was still connected/open on my laptop from testing last night. In the middle of studying today I thought I was hacked because it started typing/ch...,164
I want the absolute minimum of “mentor” interaction possible.,"Really, none would be my ideal. This is the one aspect of the WGU model I'm not thrilled about going in. Traditional university, at least the one I have experience with, the only required advisor co...",160


In [30]:
# %%
from IPython.display import display, HTML, Markdown
import pandas as pd

def make_details(text, front=100):
    if not isinstance(text, str) or len(text) <= front:
        return text
    summary = text[:front].replace('\n',' ') + "…"
    return f"<details><summary>{summary}</summary><div>{text}</div></details>"

# Compute total length
df['total_length'] = df['title'].str.len() + df['selftext'].str.len()
top_length = df.nlargest(10, 'total_length')

# Build DataFrame with HTML details cells
table_length = pd.DataFrame({
    'Title':        top_length['title'].map(lambda t: make_details(t, 50)),
    'Self-text':    top_length['selftext'].map(lambda t: make_details(t, 150)),
    'Total Length': top_length['total_length'],
})

# Render HTML (preserving our <details> tags) with wrapping CSS
html = table_length.to_html(
    index=False,
    escape=False,
    formatters={'Total Length': '{:,.0f}'.format},
    classes='truncate-table'
)
css = """
<style>
.truncate-table td { white-space: normal; }
.truncate-table th, .truncate-table td { padding: 4px; text-align: left; }
details summary { cursor: pointer; font-weight: bold; margin-bottom:4px; }
</style>
"""

# Header + note
display(Markdown("## Longest Posts\n\n_Note: use the arrow toggle to show the full text_"))
display(HTML(css + html))

## Longest Posts

_Note: use the arrow toggle to show the full text_

In [2]:
# filename: analyze_post_coverage_text.py

import sqlite3
import pandas as pd

# Connect and load timestamps
conn = sqlite3.connect('/Users/buddy/Desktop/WGU-Reddit/db/WGU-Reddit.db')
df = pd.read_sql_query("SELECT created_utc FROM posts", conn)
df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')

# Coverage summary
start = df['created_utc'].min()
end = df['created_utc'].max()
total = len(df)

print("🕰️ Coverage Summary")
print(f"Total posts: {total}")
print(f"Earliest post: {start}")
print(f"Latest post:   {end}\n")

# Group by week
print("📆 Weekly Post Counts (YYYY-WW):")
weekly = df.set_index('created_utc').resample('W').size()
for ts, count in weekly.items():
    iso = ts.strftime("%Y-W%V")
    print(f"{iso}: {count}")

🕰️ Coverage Summary
Total posts: 17213
Earliest post: 2014-12-28 18:26:32
Latest post:   2025-06-23 02:46:01

📆 Weekly Post Counts (YYYY-WW):
2014-W52: 1
2015-W01: 1
2015-W02: 0
2015-W03: 2
2015-W04: 0
2015-W05: 1
2015-W06: 1
2015-W07: 0
2015-W08: 0
2015-W09: 0
2015-W10: 1
2015-W11: 0
2015-W12: 0
2015-W13: 0
2015-W14: 0
2015-W15: 0
2015-W16: 0
2015-W17: 0
2015-W18: 0
2015-W19: 0
2015-W20: 1
2015-W21: 0
2015-W22: 0
2015-W23: 1
2015-W24: 0
2015-W25: 1
2015-W26: 0
2015-W27: 0
2015-W28: 0
2015-W29: 0
2015-W30: 0
2015-W31: 1
2015-W32: 0
2015-W33: 0
2015-W34: 0
2015-W35: 3
2015-W36: 0
2015-W37: 0
2015-W38: 0
2015-W39: 0
2015-W40: 0
2015-W41: 0
2015-W42: 6
2015-W43: 5
2015-W44: 1
2015-W45: 5
2015-W46: 3
2015-W47: 3
2015-W48: 1
2015-W49: 3
2015-W50: 0
2015-W51: 0
2015-W52: 0
2016-W53: 1
2016-W01: 1
2016-W02: 0
2016-W03: 1
2016-W04: 0
2016-W05: 0
2016-W06: 0
2016-W07: 2
2016-W08: 0
2016-W09: 0
2016-W10: 1
2016-W11: 1
2016-W12: 1
2016-W13: 0
2016-W14: 0
2016-W15: 1
2016-W16: 1
2016-W17: 0
2016-W